# 価値関数

```{math}
\newcommand{\E}{{\mathrm E}}
\newcommand{\underE}[2]{\underset{\begin{subarray}{c}#1 \end{subarray}}{\E}\left[ #2 \right]}
\newcommand{\Epi}[1]{\underset{\begin{subarray}{c}\tau \sim \pi \end{subarray}}{\E}\left[ #1 \right]}
```

本章では，強化学習理論の基礎となる，価値関数/行動価値関数というものについて見ていきます．価値関数/行動価値関数とは直感的に捉えると，ある状態$s$や状態行動対$(s,a)$が，どのくらいの価値を持つかを表すものです．価値関数は強化学習アルゴリズムの目的である"最適方策"と密接な関わりがあり，重要な概念です．ここでは，具体的に以下の4点について解説していきます．

- 価値関数/行動価値関数の定義
- これらを用いた最適方策の定義
- 価値関数の性質
- 価値関数の推定

## 定義

まずは，一般的に用いられる価値関数/行動価値関数の定義を確認します．

- 価値関数(Value Function)

  $$V^\pi(s) = \underE{\pi}{\lim_{T \rightarrow \infty} \sum_{t=0}^T \gamma^t R_t | S_0 = s}$$

- 行動価値関数(Action Value Function)

  $$Q^\pi(s,a) = \underE{\pi}{\lim_{T \rightarrow \infty} \sum_{t=0}^T \gamma^t R_t | S_0 = s, A_0 = a}$$

価値関数$V^\pi(s)$は「ある状態$s$から方策$\pi$に従って行動した場合の得られる期待割引累積報酬」と解釈できます．
また，行動価値関数は「ある状態$s$で，ある行動$a$を取った後，方策$\pi$に従って行動した場合の得られる期待割引累積報酬」と解釈できます．
２つの違いは，価値関数$V^\pi(s)$は最初の行動$A_0$が方策に従ってサンプルされるのに対して，行動価値関数$Q^\pi(s,a)$では$A_0$が引数によって与えられるという点です．なので，2つの関数には以下の関係が成り立ちます．

$$V^\pi(s) = \underE{a \sim \pi}{Q^\pi(s, a)}$$

## 価値関数/行動価値関数の推定(モンテカルロ法)

価値関数/行動価値関数の定義から，

  1. 確率変数である$\sum \gamma^t R_t$を実際にサンプルする．
  2. サンプルされたデータを元に，$V^\pi(s), Q^\pi(s, a)$を推定する．

という非常にシンプルな推定方法を考えることが出来ます．
このようなコンセプトを持った手法はモンテカルロ法と言われています．

実際にモンテカルロ法をシミュレーションに適用する様子を見ていきましょう．

### 実験設定

環境として，OpenAI Gymの[FrozenLake](https://www.gymlibrary.ml/environments/toy_text/frozen_lake/)を用います．
環境に関する設定は，ドキュメントを参照してください．

In [1]:
import gym

env = gym.make(
    "FrozenLake-v1",
    desc=["SFFF", "FHFH", "FFFH", "HFFG"],
    map_name="4x4",
    is_slippery=False,
)

価値推定の対象となる方策は，ランダム方策とします

In [ ]:
def policy(state):
    return env.action_space.sample()  # 行動を行動空間からランダムにサンプルする


次に，各パラメータを設定します．
環境から状態数と行動数を取得し，割引率$\gamma = 0.99$と設定します

In [ ]:
state_size = env.observation_space.n
action_size = env.action_space.n
GAMMA = 0.99


環境に対して，方策を用いて，経験を集める関数を定義します．
ここで経験とは，

$$[(s_0, a_0, s_{t+1})]$$


In [ ]:
from dataclasses import dataclass


@dataclass
class Transition:
    state: int
    next_state: int
    action: int
    reward: float


In [ ]:
from typing import Callable


def generate_episode(env: gym.Env, policy: Callable):
    episode: list[Transition] = []
    state = env.reset()
    while True:
        action = policy(state)
        next_state, reward, done, _ = env.step(action)
        episode.append(Transition(state, next_state, action, reward))
        if done:
            break
        else:
            state = next_state

    return episode
